<a href="https://colab.research.google.com/github/lutakrystal305/RecognitionFace/blob/main/MTCNN_Facenet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
%cd "/content/gdrive/MyDrive/MTCNN_FACENET"

/content/gdrive/MyDrive/MTCNN_FACENET


In [ ]:
import sklearn

#print('The nltk version is {}.'.format(nltk.__version__))
print('The scikit-learn version is {}.'.format(sklearn.__version__))


The scikit-learn version is 0.24.1.


In [ ]:
pathColab = "/content/gdrive/MyDrive/MTCNN_FACENET"

In [ ]:
from src import facenet
from src.align import detect_face
from sklearn.svm import SVC
import pickle
import numpy as np
import cv2
import collections
import tensorflow as tf

# import dependencies
from IPython.display import display, Javascript, Image
from google.colab.output import eval_js
from google.colab.patches import cv2_imshow
from base64 import b64decode, b64encode
#import cv2
#import numpy as np
import PIL
import io
import html
import time
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
#parser = argparse.ArgumentParser()
#parser.add_argument('--path', help='Path of the video you want to test on.', default=0)
#args = parser.parse_args()

MINSIZE = 20
THRESHOLD = [0.6, 0.7, 0.7] #[0.6, 0.7, 0.7]
FACTOR = 0.709
IMAGE_SIZE = 182
INPUT_IMAGE_SIZE = 160
CLASSIFIER_PATH = 'Models/facemodel.pkl'
#VIDEO_PATH = args.path
FACENET_MODEL_PATH = 'Models/20180402-114759.pb'


In [ ]:
with open(CLASSIFIER_PATH, 'rb') as file:
    model, class_names = pickle.load(file)
print("Custom Classifier, Successfully loaded")

Custom Classifier, Successfully loaded


In [ ]:
#WEBCAM
# function to convert the JavaScript object into an OpenCV image
def js_to_image(js_reply):
  """
  Params:
          js_reply: JavaScript object containing image from webcam
  Returns:
          img: OpenCV BGR image
  """
  # decode base64 image
  image_bytes = b64decode(js_reply.split(',')[1])
  # convert bytes to numpy array
  jpg_as_np = np.frombuffer(image_bytes, dtype=np.uint8)
  # decode numpy array into OpenCV BGR image
  img = cv2.imdecode(jpg_as_np, flags=1)

  return img

# function to convert OpenCV Rectangle bounding box image into base64 byte string to be overlayed on video stream
def bbox_to_bytes(bbox_array):
  """
  Params:
          bbox_array: Numpy array (pixels) containing rectangle to overlay on video stream.
  Returns:
        bytes: Base64 image byte string
  """
  # convert array into PIL image
  bbox_PIL = PIL.Image.fromarray(bbox_array, 'RGBA')
  iobuf = io.BytesIO()
  # format bbox into png for return
  bbox_PIL.save(iobuf, format='png')
  # format return string
  bbox_bytes = 'data:image/png;base64,{}'.format((str(b64encode(iobuf.getvalue()), 'utf-8')))

  return bbox_bytes

In [ ]:
#WEBCAM 
# JavaScript to properly create our live video stream using our webcam as input
def video_stream():
  js = Javascript('''
    var video;
    var div = null;
    var stream;
    var captureCanvas;
    var imgElement;
    var labelElement;
    
    var pendingResolve = null;
    var shutdown = false;
    
    function removeDom() {
       stream.getVideoTracks()[0].stop();
       video.remove();
       div.remove();
       video = null;
       div = null;
       stream = null;
       imgElement = null;
       captureCanvas = null;
       labelElement = null;
    }
    
    function onAnimationFrame() {
      if (!shutdown) {
        window.requestAnimationFrame(onAnimationFrame);
      }
      if (pendingResolve) {
        var result = "";
        if (!shutdown) {
          captureCanvas.getContext('2d').drawImage(video, 0, 0, 640, 480);
          result = captureCanvas.toDataURL('image/jpeg', 0.8)
        }
        var lp = pendingResolve;
        pendingResolve = null;
        lp(result);
      }
    }
    
    async function createDom() {
      if (div !== null) {
        return stream;
      }

      div = document.createElement('div');
      div.style.border = '2px solid black';
      div.style.padding = '3px';
      div.style.width = '100%';
      div.style.maxWidth = '600px';
      document.body.appendChild(div);
      
      const modelOut = document.createElement('div');
      modelOut.innerHTML = "<span>Status:</span>";
      labelElement = document.createElement('span');
      labelElement.innerText = 'No data';
      labelElement.style.fontWeight = 'bold';
      modelOut.appendChild(labelElement);
      div.appendChild(modelOut);
           
      video = document.createElement('video');
      video.style.display = 'block';
      video.width = div.clientWidth - 6;
      video.setAttribute('playsinline', '');
      video.onclick = () => { shutdown = true; };
      stream = await navigator.mediaDevices.getUserMedia(
          {video: { facingMode: "environment"}});
      div.appendChild(video);

      imgElement = document.createElement('img');
      imgElement.style.position = 'absolute';
      imgElement.style.zIndex = 1;
      imgElement.onclick = () => { shutdown = true; };
      div.appendChild(imgElement);
      
      const instruction = document.createElement('div');
      instruction.innerHTML = 
          '<span style="color: red; font-weight: bold;">' +
          'When finished, click here or on the video to stop this demo</span>';
      div.appendChild(instruction);
      instruction.onclick = () => { shutdown = true; };
      
      video.srcObject = stream;
      await video.play();

      captureCanvas = document.createElement('canvas');
      captureCanvas.width = 640; //video.videoWidth;
      captureCanvas.height = 480; //video.videoHeight;
      window.requestAnimationFrame(onAnimationFrame);
      
      return stream;
    }
    async function stream_frame(label, imgData) {
      if (shutdown) {
        removeDom();
        shutdown = false;
        return '';
      }

      var preCreate = Date.now();
      stream = await createDom();
      
      var preShow = Date.now();
      if (label != "") {
        labelElement.innerHTML = label;
      }
            
      if (imgData != "") {
        var videoRect = video.getClientRects()[0];
        imgElement.style.top = videoRect.top + "px";
        imgElement.style.left = videoRect.left + "px";
        imgElement.style.width = videoRect.width + "px";
        imgElement.style.height = videoRect.height + "px";
        imgElement.src = imgData;
      }
      
      var preCapture = Date.now();
      var result = await new Promise(function(resolve, reject) {
        pendingResolve = resolve;
      });
      shutdown = false;
      
      return {'create': preShow - preCreate, 
              'show': preCapture - preShow, 
              'capture': Date.now() - preCapture,
              'img': result};
    }
    ''')

  display(js)
  
def video_frame(label, bbox):
  data = eval_js('stream_frame("{}", "{}")'.format(label, bbox))
  return data

In [ ]:
!pip install scikit-learn==0.24.1 # Đồng bộ với phiên bản trên máy để chạy được model SVM

     |████████████████████████████████| 22.3MB 1.3MB/s 
  Found existing installation: scikit-learn 0.22.2.post1
    Uninstalling scikit-learn-0.22.2.post1:
      Successfully uninstalled scikit-learn-0.22.2.post1


In [ ]:
#WEBCAM
with tf.Graph().as_default():

    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.6)
    sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options, log_device_placement=False))

    with sess.as_default():

        # Load the model
        print('Loading feature extraction model')
        facenet.load_model(FACENET_MODEL_PATH)

        # Get input and output tensors
        images_placeholder = tf.compat.v1.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.compat.v1.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.compat.v1.get_default_graph().get_tensor_by_name("phase_train:0")
        embedding_size = embeddings.get_shape()[1]

        pnet, rnet, onet = detect_face.create_mtcnn(sess, "src/align")

        people_detected = set()
        person_detected = collections.Counter()

        #cap  = VideoStream(src=0).start()
        # start streaming video from webcam #--------- NEW(WEBCAM)
        video_stream()  #--------- NEW(WEBCAM)
        # label for video   #--------- NEW(WEBCAM)
        label_html = 'Capturing...'   #--------- NEW(WEBCAM)
        # initialze bounding box to empty   #--------- NEW(WEBCAM)
        bbox = ''
        count = 0 
        while (True):
            #frame = cap.read()
            #frame = cv2.imread('src/img/7.jpg')
            #print(frame.shape)
            #frame = cv2.resize(frame, dsize=(600, 400))
            #frame = imutils.resize(frame, width=600)
            #frame = cv2.flip(frame, 1)

            js_reply = video_frame(label_html, bbox)    #------- NEW(WEBCAM)
            if not js_reply:      #------- NEW(WEBCAM)
                break     #------- NEW(WEBCAM)

            # create transparent overlay for bounding box  -------- NEW (WEBCAM)
            bbox_array = np.zeros([480,640,4], dtype=np.uint8)

            # convert JS response to OpenCV Image     #------- NEW(WEBCAM)
            frame = js_to_image(js_reply["img"])      #------- NEW(WEBCAM)

            bounding_boxes, _ = detect_face.detect_face(frame, MINSIZE, pnet, rnet, onet, THRESHOLD, FACTOR)

            faces_found = bounding_boxes.shape[0]
            print(faces_found)
            if faces_found > 0:
                det = bounding_boxes[:, 0:4]
                bb = np.zeros((faces_found, 4), dtype=np.int32)
                for i in range(faces_found):
                    bb[i][0] = det[i][0]
                    bb[i][1] = det[i][1]
                    bb[i][2] = det[i][2]
                    bb[i][3] = det[i][3]

                    # Cat phan khuon mat tim duoc
                    cropped = frame[bb[i][1]:bb[i][3], bb[i][0]:bb[i][2], :]
                    scaled = cv2.resize(cropped, (INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE),
                                                interpolation=cv2.INTER_CUBIC)
                    scaled = facenet.prewhiten(scaled)
                    scaled_reshape = scaled.reshape(-1, INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE, 3)
                    feed_dict = {images_placeholder: scaled_reshape, phase_train_placeholder: False}
                    emb_array = sess.run(embeddings, feed_dict=feed_dict)
                        
                    # Dua vao model de classifier
                    predictions = model.predict_proba(emb_array)
                    best_class_indices = np.argmax(predictions, axis=1)
                    best_class_probabilities = predictions[
                    np.arange(len(best_class_indices)), best_class_indices]
                            
                    # Lay ra ten va ty le % cua class co ty le cao nhat
                    best_name = class_names[best_class_indices[0]]
                    print("Name: {}, Probability: {}".format(best_name, best_class_probabilities))

                    # Ve khung mau xanh quanh khuon mat
                    bbox_array = cv2.rectangle(bbox_array, (bb[i][0], bb[i][1]), (bb[i][2], bb[i][3]), (0, 255, 0), 2) # ---- NEW(WEBCAM)
                    # cv2.rectangle(frame, (bb[i][0], bb[i][1]), (bb[i][2], bb[i][3]), (0, 255, 0), 2)
                    text_x = bb[i][0]
                    text_y = bb[i][3] + 20

                    # Neu ty le nhan dang > 0.5 thi hien thi ten
                    if best_class_probabilities > 0.7:
                        name = class_names[best_class_indices[0]]
                    else:
                        # Con neu <=0.5 thi hien thi Unknow
                        name = "Unknown"
                                
                        # Viet text len tren frame    
                        # Viet text len tren frame    
                    bbox_array = cv2.putText(bbox_array, name, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX_SMALL,  #-------  NEW(WEBCAM)
                                        1, (255, 255, 255), thickness=1, lineType=2)
                    bbox_array = cv2.putText(bbox_array, str(round(best_class_probabilities[0], 3)), (text_x, text_y + 17), # ----- NEW(WEBCAM)
                                        cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                        1, (255, 255, 255), thickness=1, lineType=2)
                        #cv2.putText(frame, name, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                        #1, (255, 255, 255), thickness=1, lineType=2)
                        #cv2.putText(frame, str(round(best_class_probabilities[0], 3)), (text_x, text_y + 17),
                                        #cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                        #1, (255, 255, 255), thickness=1, lineType=2)
                    person_detected[best_name] += 1    
                        
                bbox_array[:,:,3] = (bbox_array.max(axis = 2) > 0 ).astype(int) * 255  #----- NEW(WEBCAM)
                    # convert overlay of bbox into bytes  
                bbox_bytes = bbox_to_bytes(bbox_array) #----- NEW(WEBCAM)
                    # update bbox so next frame gets new overlay
                bbox = bbox_bytes    #----- NEW(WEBCAM)

In [ ]:
from google.colab import files

uploaded = files.upload()

Saving kaka3.mp4 to kaka3.mp4


In [ ]:
with tf.Graph().as_default():

    gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.6)
    sess = tf.compat.v1.Session(config=tf.compat.v1.ConfigProto(gpu_options=gpu_options, log_device_placement=False))

    with sess.as_default():

        # Load the model
        print('Loading feature extraction model')
        facenet.load_model(FACENET_MODEL_PATH)

        # Get input and output tensors
        images_placeholder = tf.compat.v1.get_default_graph().get_tensor_by_name("input:0")
        embeddings = tf.compat.v1.get_default_graph().get_tensor_by_name("embeddings:0")
        phase_train_placeholder = tf.compat.v1.get_default_graph().get_tensor_by_name("phase_train:0")
        embedding_size = embeddings.get_shape()[1]

        pnet, rnet, onet = detect_face.create_mtcnn(sess, "src/align")

        people_detected = set()
        person_detected = collections.Counter()

        try:
          vid = cv2.VideoCapture(int('kaka3.mp4'))
        except:
          vid = cv2.VideoCapture('kaka3.mp4') #thay doi dia chi vid
        

        out = None
        
        width = int(vid.get(cv2.CAP_PROP_FRAME_WIDTH))
        height = int(vid.get(cv2.CAP_PROP_FRAME_HEIGHT))
        fps = int(vid.get(cv2.CAP_PROP_FPS))
        codec = cv2.VideoWriter_fourcc(*'XVID')
        out = cv2.VideoWriter('output.avi', codec, fps, (width, height))

        

        while (True):
            return_value, frame = vid.read()
            #print(frame.shape)
            if return_value:
                #frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
                #frame = cv2.resize(frame, dsize=(600, 400))
                #frame = cv2.flip(frame, 1)
                print(True)
                #image = Image.fromarray(frame)
            else:
                print('Video has ended or failed, try a different video format!')
                break          
            #frame = cv2.imread('src/img/7.jpg')
            
            #frame = imutils.resize(frame, width=600)


            bounding_boxes, _ = detect_face.detect_face(frame, MINSIZE, pnet, rnet, onet, THRESHOLD, FACTOR)

            faces_found = bounding_boxes.shape[0]
            print(faces_found)
            if faces_found > 0:
                det = bounding_boxes[:, 0:4]
                bb = np.zeros((faces_found, 4), dtype=np.int32)
                for i in range(faces_found):
                    bb[i][0] = det[i][0]
                    bb[i][1] = det[i][1]
                    bb[i][2] = det[i][2]
                    bb[i][3] = det[i][3]

                    
                    #print(bb[i][3] - bb[i][1])
                    # Cat phan khuon mat tim duoc
                    cropped = frame[bb[i][1]:bb[i][3], bb[i][0]:bb[i][2], :]
                    print(cropped)
                    if (cropped.shape[0] == 0 or cropped.shape[1] == 0):
                        cropped = frame[abs(bb[i][1]):abs(bb[i][3]), abs(bb[i][0]):abs(bb[i][2]), :]
                    print(cropped.shape[0], cropped.shape[1])
                    scaled = cv2.resize(cropped, (INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE),
                                                interpolation=cv2.INTER_CUBIC)
                    scaled = facenet.prewhiten(scaled)
                    scaled_reshape = scaled.reshape(-1, INPUT_IMAGE_SIZE, INPUT_IMAGE_SIZE, 3)
                    feed_dict = {images_placeholder: scaled_reshape, phase_train_placeholder: False}
                    emb_array = sess.run(embeddings, feed_dict=feed_dict)
                        
                    # Dua vao model de classifier
                    predictions = model.predict_proba(emb_array)
                    best_class_indices = np.argmax(predictions, axis=1)
                    best_class_probabilities = predictions[
                    np.arange(len(best_class_indices)), best_class_indices]
                            
                    # Lay ra ten va ty le % cua class co ty le cao nhat
                    best_name = class_names[best_class_indices[0]]
                    print("Name: {}, Probability: {}".format(best_name, best_class_probabilities))

                    cv2.rectangle(frame, (bb[i][0], bb[i][1]), (bb[i][2], bb[i][3]), (0, 255, 0), 2)
                    text_x = bb[i][0]
                    text_y = bb[i][3] + 20

                    # Neu ty le nhan dang > 0.5 thi hien thi ten
                    if best_class_probabilities > 0.7:
                        name = class_names[best_class_indices[0]]
                    else:
                        # Con neu <=0.5 thi hien thi Unknow
                        name = "Unknown"
                                
                        # Viet text len tren frame    
                        # Viet text len tren frame    
                    cv2.putText(frame, name, (text_x, text_y), cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                        1, (255, 255, 255), thickness=1, lineType=2)
                    cv2.putText(frame, str(round(best_class_probabilities[0], 3)), (text_x, text_y + 17),
                                        cv2.FONT_HERSHEY_COMPLEX_SMALL,
                                        1, (255, 255, 255), thickness=1, lineType=2)
                    person_detected[best_name] += 1    
                    
                result = np.asarray(frame)
                #result = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)

                #cv2_imshow(result)

                out.write(result)

                #if cv2.waitKey(1) & 0xFF == ord('q'): break
        cv2.destroyAllWindows()
               

Kết quả truyền trực tuyến bị cắt bớt đến 5000 dòng cuối.
  [ 31  69  90]]

 [[145 153 154]
  [147 153 154]
  [147 153 154]
  ...
  [ 45  69  92]
  [ 44  68  91]
  [ 31  69  90]]

 [[145 153 154]
  [146 152 153]
  [146 152 153]
  ...
  [ 44  68  91]
  [ 42  66  89]
  [ 29  67  88]]]
291 250
Name: BOSS, Probability: [0.78495626]
True
1
[[[144 157 159]
  [145 158 160]
  [148 160 160]
  ...
  [186 206 211]
  [190 208 213]
  [189 207 212]]

 [[143 156 158]
  [145 158 160]
  [149 161 161]
  ...
  [186 206 211]
  [190 208 213]
  [189 207 212]]

 [[139 151 151]
  [141 153 153]
  [135 144 143]
  ...
  [186 206 210]
  [189 207 211]
  [189 207 211]]

 ...

 [[ 53  51  55]
  [ 50  48  52]
  [ 45  45  49]
  ...
  [ 64  87 114]
  [ 68  89 116]
  [ 69  90 117]]

 [[ 48  48  52]
  [ 46  46  50]
  [ 43  43  47]
  ...
  [ 65  88 115]
  [ 66  89 116]
  [ 68  91 118]]

 [[ 47  47  51]
  [ 44  44  48]
  [ 43  43  47]
  ...
  [ 66  89 116]
  [ 67  90 117]
  [ 69  92 119]]]
316 268
Name: BOSS, Probability: [

In [ ]:
# define helper function to display videos
import io 
from IPython.display import HTML
from base64 import b64encode
def show_video(file_name, width=640):
  # show resulting deepsort video
  mp4 = open(file_name,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width="{0}" height="600" controls>
        <source src="{1}" type="video/mp4">
  </video>
  """.format(width, data_url))

In [ ]:
# convert resulting video from avi to mp4 file format
#import os
#path_video = os.path.join("outputs","tracker.avi")
#%cd outputs/
!ffmpeg -y -loglevel panic -i output.avi output.mp4
#%cd ..

# output object tracking video
#path_output = os.path.join("outputs","output.mp4")
show_video('output.mp4', width=960)